## Tanya Ortega 

## MADS 509

## Sept 8, 2025

# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [2]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [3]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

In [4]:

import requests, re

robots_url = "https://www.azlyrics.com/robots.txt"
resp = requests.get(robots_url, timeout=20)
print("Status:", resp.status_code)
print("---- robots.txt (first 2000 chars) ----")
print(resp.text[:2000])

# Quick read helper: list Disallow rules
disallows = re.findall(r'(?im)^\s*Disallow:\s*(\S+)', resp.text)
print("\nDisallow rules:", disallows)

Status: 200
---- robots.txt (first 2000 chars) ----
User-agent: *
Disallow: /lyricsdb/
Disallow: /song/
Disallow: /search/
Disallow: /suggest/
Allow: /

User-agent: 008
Disallow: /


Disallow rules: ['/lyricsdb/', '/song/', '/search/', '/suggest/', '/']


## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A:
The robots.txt file blocks /lyricsdb/, /song/, /search/, and /suggest/, but it doesn’t block /lyrics/ where the songs are. That means scraping the lyrics pages is allowed.

In [5]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 

In [6]:
from urllib.parse import urljoin

lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page, headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    })
    time.sleep(5 + 10*random.random())

    soup = BeautifulSoup(r.text, "html.parser")

    # grab links inside the album/song list container
    found = []
    container = soup.select_one("#listAlbum")
    if container:
        for a in container.select("a[href]"):
            href = a.get("href", "")
            if "lyrics/" in href and href.endswith(".html"):
                full_url = urljoin(artist_page, href)  # make absolute
                found.append(full_url)

    # de-dupe while keeping order
    seen = set()
    for u in found:
        if u not in seen:
            seen.add(u)
            lyrics_pages[artist].append(u)

    print(f"{artist}: found {len(lyrics_pages[artist])} songs")
    print("  sample:", lyrics_pages[artist][:5])

robyn: found 104 songs
  sample: ['https://www.azlyrics.com/lyrics/robyn/bumpyride.html', 'https://www.azlyrics.com/lyrics/robyn/inmyheart.html', 'https://www.azlyrics.com/lyrics/robyn/youvegotthatsomething.html', 'https://www.azlyrics.com/lyrics/robyn/doyouknowwhatittakes.html', 'https://www.azlyrics.com/lyrics/robyn/thelasttime.html']
cher: found 334 songs
  sample: ['https://www.azlyrics.com/lyrics/cher/allireallywanttodo.html', 'https://www.azlyrics.com/lyrics/cher/igotosleep.html', 'https://www.azlyrics.com/lyrics/cher/needlesandpins.html', 'https://www.azlyrics.com/lyrics/cher/dontthinktwice.html', 'https://www.azlyrics.com/lyrics/cher/shethinksistillcare.html']


In [7]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20)

In [8]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For robyn we have 104.
The full pull will take for this artist will take 0.29 hours.
For cher we have 334.
The full pull will take for this artist will take 0.93 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [12]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [18]:
# ---- Save EXACTLY 20 songs per artist (Robyn & Cher) ----
import os, shutil, time, random, requests
from bs4 import BeautifulSoup

# 0) fresh output folder
if os.path.isdir("lyrics"):
    shutil.rmtree("lyrics")
os.mkdir("lyrics")

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.azlyrics.com/"
}

total_saved = 0
for artist, all_links in lyrics_pages.items():
    # 1) pick 20 links
    if len(all_links) < 20:
        raise AssertionError(f"{artist}: only {len(all_links)} links available (<20).")
    # Use the first 20; for a random 20 use: random.sample(all_links, 20)
    links = all_links[:20]

    # 2) artist subfolder
    outdir = os.path.join("lyrics", artist)
    os.makedirs(outdir, exist_ok=True)

    # 3) fetch & save
    saved = 0
    for link in links:
        try:
            r = requests.get(link, headers=HEADERS, timeout=30)
            time.sleep(5 + 10*random.random())
            if r.status_code != 200:
                continue

            soup = BeautifulSoup(r.text, "html.parser")

            # title (best-effort)
            title = ""
            tb = soup.find("div", class_="col-xs-12 col-lg-8 text-center")
            if tb:
                b = tb.find("b")
                if b:
                    title = b.get_text(strip=True)

            # lyrics: first div with no id/class and enough text
            lyr_div = None
            for d in soup.find_all("div"):
                if not d.get("id") and not d.get("class"):
                    t = d.get_text("\n", strip=True)
                    if len(t) > 200:
                        lyr_div = d
                        break
            if not lyr_div:
                continue

            fname = generate_filename_from_link(link)  # uses your prof's helper
            with open(os.path.join(outdir, fname), "w", encoding="utf-8") as f:
                f.write((title or "") + "\n\n")
                f.write(lyr_div.get_text("\n", strip=True))

            saved += 1
            total_saved += 1
            if saved == 20:  # stop at exactly 20
                break
        except Exception as e:
            print(f"Skip {link} -> {e}")

    print(f"{artist}: saved {saved} songs (target = 20)")
    assert saved == 20, f"{artist}: saved {saved}, expected 20."

print(f"Total pages saved: {total_saved} (should be 40)")

robyn: saved 20 songs (target = 20)
cher: saved 20 songs (target = 20)
Total pages saved: 40 (should be 40)


In [19]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 1.47 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [20]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [21]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")

For cher we have 20 files.
For cher we have roughly 3837 words, 732 are unique.
For robyn we have 20 files.
For robyn we have roughly 4665 words, 673 are unique.
